In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm as tqdm
from multiprocessing import Pool
import time,gc,sys

In [2]:
# f22:上一天该站台最常见出站台
# f23:上一天该站台全部站出站台人数
# f24:上一天该站台全部站进站台人数

In [3]:
# 统计进出站每天的变化情况
def get_daily_activity(i):
    if i<10:
        s = '0'+str(i)
    else:
        s = str(i)
    day_activ = pd.read_csv('data/Metro_train/record_2019-01-%s.csv'%s)
    day_activ['time'] = pd.to_datetime(day_activ.time)
    day_activ['stationID'] = day_activ['stationID']
    day_activ['deviceID'] = day_activ['deviceID']
    day_activ['status'] = day_activ['status']
    day_activ['payType'] = day_activ['payType']
    day_activ['lineID'] = day_activ['lineID'].apply(lambda x: {'A':0,'B':2,'C':1}[x])
    return day_activ

def construct_df():
    final = pd.DataFrame()
    for i in range(81):
        temp = pd.DataFrame()
        temp['minute'] = list(range(1440))
        temp['stationID'] = i
        final = final.append(temp)
    final['minute'] = final['minute']/10
    return final


In [4]:
all_df = pd.DataFrame()

#for date in range(7,26):
for date in list(range(1,26))+[28]:

    final = construct_df()
    day_activ = get_daily_activity(date)
    
    df = pd.DataFrame()
    for i in range(10):
        day_activ['minute'] = (day_activ['time'].dt.minute-i+day_activ['time'].dt.hour*60)//10+0.1*i
        day_activ['minute'] = day_activ['minute'].apply(lambda x:143+1+x if x<0 else x)
        df = df.append(day_activ)
    df.sort_values(by='minute',inplace=True)
    come = df
    # 上一天该时刻进站分布
    for i in tqdm(range(81)):
        f = come[(come.stationID==i)&(come.status==1)].groupby('minute',as_index=False).agg({'status':'count'}).rename(columns={'status':'%d_come'%i})
        final = final.merge(f,on='minute',how='left')
    # 出站方差分布，最大值，平均值 
    temp_name = ['%d_come'%i for i in range(81)]
    final[temp_name].fillna(0,inplace=True)
    final['come_max'] = final[temp_name].max(axis=1)
    final['come_mean'] = final[temp_name].mean(axis=1)
    final['come_var'] = final[temp_name].var(axis=1)
    final['come_max_id'] = final[temp_name].idxmax(axis=1)

    # 上一天该时刻出站分布
    for i in tqdm(range(81)):
        f = come[(come.stationID==i)&(come.status==0)].groupby('minute',as_index=False).agg({'status':'count'}).rename(columns={'status':'%d_out'%i})
        final = final.merge(f,on='minute',how='left')
    # 出站方差分布，最大值，平均值 
    final[temp_name].fillna(0,inplace=True)

    temp_name = ['%d_out'%i for i in range(81)]
    final['out_max'] = final[temp_name].max(axis=1)
    final['out_mean'] = final[temp_name].mean(axis=1)
    final['out_var'] = final[temp_name].var(axis=1)
    final['out_max_id'] = final[temp_name].idxmax(axis=1)

    # 上一天该时刻该站出站分布
    day_activ = get_daily_activity(date)
    day_activ['minute'] = (day_activ['time'].dt.minute+day_activ['time'].dt.hour*60)//3

    come = day_activ[day_activ.status==1]
    out = day_activ[day_activ.status==0][['userID','stationID','minute']].rename(columns={'stationID':'out_station'})
    temp = pd.DataFrame()
    for i in tqdm(range(480)):
        temp_come = come[come.minute==i]
        temp_out = out[out.minute>i][['userID','out_station']]
        temp = temp.append(temp_come.merge(temp_out,on='userID',how='left').drop_duplicates(subset=['time','stationID','userID'],keep='first'))
    day_activ = temp
    df = pd.DataFrame()
    for i in range(10):
        day_activ['minute'] = (day_activ['time'].dt.minute-i+day_activ['time'].dt.hour*60)//10+0.1*i
        day_activ['minute'] = day_activ['minute'].apply(lambda x:143+1+x if x<0 else x)
        df = df.append(day_activ)
    df.sort_values(by='minute',inplace=True)
    come = df

    come_out_dist = come.groupby(['stationID','minute','out_station'],as_index=False).agg({'status':'count'}).rename(columns={'status':'out_number'})
    for i in range(81):
        f = come_out_dist[come_out_dist.out_station==i][['stationID','minute','out_number']].rename(columns={'out_number':'%d_come_out'%i})
        final = final.merge(f,on=['stationID','minute'],how='left')
    # 出站方差分布，最大值，平均值 
    temp_name = ['%d_come_out'%i for i in range(81)]
    final[temp_name].fillna(0,inplace=True)

    final['come_out_max'] = final[temp_name].max(axis=1)
    final['come_out_mean'] = final[temp_name].mean(axis=1)
    final['come_out_var'] = final[temp_name].var(axis=1)
    final['come_out_max_id'] = final[temp_name].idxmax(axis=1)
    
    # 上一天该时刻该站进站分布
    day_activ = get_daily_activity(date)
    day_activ['minute'] = (day_activ['time'].dt.minute+day_activ['time'].dt.hour*60)//3

    come = day_activ[day_activ.status==1][['userID','stationID','minute']].rename(columns={'stationID':'come_station'})
    out = day_activ[day_activ.status==0]
    temp = pd.DataFrame()
    for i in tqdm(range(480)):
        temp_out = out[out.minute==i]
        temp_come = come[come.minute<i][['userID','come_station']]
        temp = temp.append(temp_out.merge(temp_come,on='userID',how='left').drop_duplicates(subset=['time','stationID','userID'],keep='first'))
    day_activ = temp
    df = pd.DataFrame()
    for i in range(10):
        day_activ['minute'] = (day_activ['time'].dt.minute-i+day_activ['time'].dt.hour*60)//10+0.1*i
        day_activ['minute'] = day_activ['minute'].apply(lambda x:143+1+x if x<0 else x)
        df = df.append(day_activ)
    df.sort_values(by='minute',inplace=True)
    out = df

    out_come_dist = out.groupby(['stationID','minute','come_station'],as_index=False).agg({'status':'count'}).rename(columns={'status':'come_number'})
    for i in range(81):
        f = out_come_dist[out_come_dist.come_station==i][['stationID','minute','come_number']].rename(columns={'come_number':'%d_out_come'%i})
        final = final.merge(f,on=['stationID','minute'],how='left')
    # 出站方差分布，最大值，平均值 
    temp_name = ['%d_come_out'%i for i in range(81)]
    final[temp_name].fillna(0,inplace=True)
    final['out_come_max'] = final[temp_name].max(axis=1)
    final['out_come_mean'] = final[temp_name].mean(axis=1)
    final['out_come_var'] = final[temp_name].var(axis=1)
    final['out_come_max_id'] = final[temp_name].idxmax(axis=1)

    # 数据格式减少内存
    final['date'] = date
    final.fillna(0,inplace=True)
    for c in final.dtypes.index:
        if final.dtypes[c] == 'float64':
            final[c] = final[c].astype('float32')
    all_df = all_df.append(final)
    print(date)
    #break

100%|██████████| 81/81 [00:15<00:00,  4.68it/s]
/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
  8%|▊         | 40/480 [00:19<03:35,  2.04it/s]/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
100%|██████████| 480/480 [02:03<00:00,  1.70it/s]


1


100%|██████████| 480/480 [02:10<00:00,  1.87it/s]


2


100%|██████████| 480/480 [02:04<00:00,  1.99it/s]


...

24


100%|██████████| 480/480 [01:56<00:00,  1.86it/s]


25


100%|██████████| 480/480 [02:19<00:00,  1.54it/s]


28


In [5]:
all_df.to_pickle('features/progress5.pkl')